In [186]:
"""
Home Win modelling

create predictions and for the hockey data

input:
output:
"""

credential_keys = '/Users/antoinetl/Documents/code/Google Keys/My First Project-4938b2ab0dc6.json'

In [187]:
# Import the important packages
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_columns', None)

In [188]:
# Load the data

# Connexion GBQ
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    credential_keys,
    )
pandas_gbq.context.credentials = credentials
# https://pandas-gbq.readthedocs.io/en/latest/intro.html

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "rational-world-288611"


sql = """
SELECT *
from `My_dataset.home_win_df`
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████| 7403/7403 [00:00<00:00, 7463.11rows/s]


In [189]:
#df = df.set_index('gameID')
df.head()

,gameID,date,teams_home_team_id,teams_away_team_id,Home_win,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
0,2014020962,2015-03-05 00:00:00+00:00,53,23,1.0,1.046875,1.761905,-0.715030,0.0,1.8,-1.8
1,2008020472,2008-12-19 00:00:00+00:00,20,16,0.0,1.781250,1.896552,-0.115302,2.0,2.0,0.0
2,2008021064,2009-03-20 00:00:00+00:00,12,2,1.0,1.638889,1.142857,0.496032,2.0,2.0,0.0
3,2009020393,2009-12-01 00:00:00+00:00,28,9,1.0,2.071429,1.791667,0.279762,2.0,2.0,0.0
4,2010021049,2011-03-15 00:00:00+00:00,25,28,0.0,1.724638,1.785714,-0.061077,2.0,2.0,0.0


In [190]:
df.describe(include='all')

,gameID,date,teams_home_team_id,teams_away_team_id,Home_win,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
count,7.403000e+03,7403,7403.000000,7403.000000,7403.000000,7403.000000,7403.000000,7403.000000,7403.000000,7403.000000,7403.000000
unique,NaN,1069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2012-04-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,2008-11-09 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2016-04-10 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2.011484e+09,NaN,16.605430,16.490207,0.548967,1.617626,1.623640,-0.006014,1.611590,1.627705,-0.016115
std,2.364136e+06,NaN,10.572126,10.512483,0.497630,0.304493,0.302217,0.433721,0.482225,0.488211,0.696741
min,2.008020e+09,NaN,1.000000,1.000000,0.000000,0.526316,0.555556,-1.698830,0.000000,0.000000,-2.500000
25%,2.009021e+09,NaN,8.000000,8.000000,0.000000,1.418762,1.428571,-0.298559,1.300000,1.300000,-0.500000


In [191]:
y = df['Home_win']

#X_dum1 = pd.get_dummies(mod_input['season'], prefix='season')
#X_dum2 = pd.get_dummies(mod_input['teams.away.team.name'], prefix='team.away')
#X_dum3 = pd.get_dummies(mod_input['teams.home.team.name'], prefix='team.home')

#X = mod_input.drop(columns=['gameType','game.homewin', 'Unnamed: 0','season','teams.away.team.name','teams.home.team.name','teams.home.leagueRecord.gameplayed','teams.away.leagueRecord.gameplayed'])
X = df.drop(columns=['Home_win','gameID', 'teams_home_team_id', 'teams_away_team_id', 'date'])
#X = pd.concat([X, X_dum1, X_dum2, X_dum3], axis=1)

In [192]:
X

,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
0,1.046875,1.761905,-0.715030,0.0,1.8,-1.8
1,1.781250,1.896552,-0.115302,2.0,2.0,0.0
2,1.638889,1.142857,0.496032,2.0,2.0,0.0
3,2.071429,1.791667,0.279762,2.0,2.0,0.0
4,1.724638,1.785714,-0.061077,2.0,2.0,0.0
...,...,...,...,...,...,...
7398,1.720000,1.666667,0.053333,1.8,1.8,0.0
7399,1.512821,1.564103,-0.051282,1.8,1.8,0.0
7400,1.562500,1.820000,-0.257500,1.8,1.8,0.0
7401,1.741935,1.786885,-0.044950,1.8,1.8,0.0


In [193]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [194]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)  
X_train = scaler.transform(X_train) # only compute mean and std here
X_test = scaler.transform(X_test)

In [195]:
#Import svm model
#from sklearn import svm

#Create a svm Classifier
#clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
#clf.fit(X_train, y_train)

#Predict the response for test dataset
#y_pred = clf.predict(X_test)

In [196]:
clf=RandomForestClassifier(n_estimators=1000)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [197]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import GridSearchCV
# import numpy as np

# parameters = {
#     "loss":["deviance"],
#     "learning_rate": [0.01, 0.05, 0.1, 0.2],
#     "min_samples_split": np.linspace(0.1, 0.5, 10),
#     "min_samples_leaf": np.linspace(0.1, 0.5, 10),
#     "max_depth":[3,8],
#     "max_features":["sqrt"],
#     "subsample":[0.5, 0.8, 0.9, 1.0],
#     "n_estimators":[10]
#     }

# clf = GridSearchCV(GradientBoostingClassifier(), 
#                    parameters, 
#                    cv=5, 
#                    n_jobs=-1, 
#                    scoring='accuracy')

# clf.fit(X_train, y_train)
# print(clf.score(X_train, y_train))
# print(clf.best_params_)
# y_pred=clf.predict(X_test)

In [198]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5411976587122918


In [199]:
#feature_importances = pd.DataFrame(clf.feature_importances_,
#                                   index = X_train.columns,
#                                   columns=['importance']).sort_values('importance', ascending=False)

In [200]:
#print(feature_importances.head(10))

In [201]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.47      0.42      0.44       964
         1.0       0.59      0.64      0.61      1257

    accuracy                           0.54      2221
   macro avg       0.53      0.53      0.53      2221
weighted avg       0.54      0.54      0.54      2221



In [202]:
y_pred=clf.predict(X)

In [203]:
y_pred.shape

(7403,)

In [204]:
df.shape

(7403, 11)

In [212]:
type(y_pred)

numpy.ndarray

In [213]:
type(df)

pandas.core.frame.DataFrame

In [214]:
y_pred_df = pd.DataFrame(data=y_pred.flatten())

In [242]:
y_pred_df.head()
y_pred_df.columns = ['Pred']

In [243]:
y_pred_df.shape

(7403, 1)

In [244]:
df.head()

,gameID,date,teams_home_team_id,teams_away_team_id,Home_win,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
0,2014020962,2015-03-05 00:00:00+00:00,53,23,1.0,1.046875,1.761905,-0.715030,0.0,1.8,-1.8
1,2008020472,2008-12-19 00:00:00+00:00,20,16,0.0,1.781250,1.896552,-0.115302,2.0,2.0,0.0
2,2008021064,2009-03-20 00:00:00+00:00,12,2,1.0,1.638889,1.142857,0.496032,2.0,2.0,0.0
3,2009020393,2009-12-01 00:00:00+00:00,28,9,1.0,2.071429,1.791667,0.279762,2.0,2.0,0.0
4,2010021049,2011-03-15 00:00:00+00:00,25,28,0.0,1.724638,1.785714,-0.061077,2.0,2.0,0.0


In [245]:
predictions = pd.concat([df, y_pred_df], axis=1)

In [246]:
#predictions.columns[-1] = 'Antoine'
#gapminder.rename(columns={0}) = 'Antoine'

y_pred_df.head()

,Pred
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [248]:
final_predictions_gcp = predictions[['date', 'teams_home_team_id', 'teams_away_team_id', 'Pred', 'Home_win']]

In [250]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "rational-world-288611"

# TODO: Set table_id to the full destination table ID (including the dataset ID).
table_id = 'My_dataset.final_pred'

pandas_gbq.to_gbq(final_predictions_gcp, table_id, project_id=project_id, if_exists='replace')

1it [00:04,  4.31s/it]


In [253]:
final_predictions_gcp.head()

,date,teams_home_team_id,teams_away_team_id,Pred,Home_win
0,2015-03-05 00:00:00+00:00,53,23,1.0,1.0
1,2008-12-19 00:00:00+00:00,20,16,1.0,0.0
2,2009-03-20 00:00:00+00:00,12,2,1.0,1.0
3,2009-12-01 00:00:00+00:00,28,9,1.0,1.0
4,2011-03-15 00:00:00+00:00,25,28,1.0,0.0
